In [2]:
# Carregamento dos dados usando pandas
import pandas as pd
from unidecode import unidecode

df_produtos = pd.read_csv('produtos_ok.csv', sep=";", encoding="latin1")
df_notas = pd.read_csv('notas_ok.csv', sep=";", encoding="latin1")

df_notas_novo = pd.read_csv('7NotasFW_ProdInfo.csv', sep=";", encoding="latin1")

In [4]:
# Variáveis que vão ser usadas ao longo de múltiplas atividades
clientes_unicos = df_notas['Cliente'].unique()
compras_unicas = df_notas['Controle'].unique()
compras_unicas_por_cliente = df_notas.groupby('Cliente')['Controle'].nunique()
compras_unicas_por_cliente_tirando_99999 = df_notas[df_notas["Cliente"] != 99999].groupby('Cliente')['Controle'].nunique()

### 2. Itens por venda: número médio de itens por venda.

In [5]:
soma = df_notas["Controle"].value_counts().sum() # Somatório de itens DIFERENTES em cada venda
media = soma/len(compras_unicas) # Dividido pela quantidade total de vendas

# Possibildiade: deveríamos considerar a quantidade interna de um item nesse somatório? (Ex: 6 unidades de waffers em uma compra devem contar como 6 ou 1? Atual: 1)
print(f"Média de itens por venda: {media} itens/venda")

# Dá pra fazer histograma com esses dados



Média de itens por venda: 9.058989203567517 itens/venda


### 3. Fidelização de clientes: número de vezes que o cliente retornou para fazer novas compras.

In [4]:
clientes_que_retornaram = compras_unicas_por_cliente[compras_unicas_por_cliente > 1] # De maneira bem simples, podemos simplesmente ver quantos vieram mais de uma vez

# Sugestões:
# 1. Ver clientes que retornaram mais vezes para ver como a fidelização realmente se aplica
clientes_que_retornaram_multiplas_vezes = compras_unicas_por_cliente[compras_unicas_por_cliente > 5]

# 2. Talvez: excluir clientes cuja primeira compra foi muito recente (pode ser um cliente novo)

# Dados
print(f"Clientes: {len(clientes_unicos)} | Retornaram: {len(clientes_que_retornaram)} | Retornaram múltiplas vezes: {len(clientes_que_retornaram_multiplas_vezes)}")
print(f"          100% | Retorna.: {round(len(clientes_que_retornaram)/len(clientes_unicos)*100, 2)}% | Retornaram múltiplas vezes: {round(len(clientes_que_retornaram_multiplas_vezes)/len(clientes_unicos)*100, 2)}%")

# Médias
soma = compras_unicas_por_cliente.sum()
soma2 = compras_unicas_por_cliente_tirando_99999.sum()
print(f"\nMédia de compras por cliente: {round(soma/len(clientes_unicos), 2)} compras/cliente")
print(f"Média de compras por cliente (excluindo o 99999): {round(soma2/len(clientes_unicos), 2)}")

# Dá pra fazer histograma com esses dados (tanto os percentuais de retorno quanto a quantidade de compras)

Clientes: 4326 | Retornaram: 3106 | Retornaram múltiplas vezes: 1928
          100% | Retorna.: 71.8% | Retornaram múltiplas vezes: 44.57%

Média de compras por cliente: 76.95 compras/cliente
Média de compras por cliente (excluindo o 99999): 21.32


### 4. Número de clientes: quantidade de pessoas que compraram no estabelecimento.

In [5]:
print(f"Clientes únicos: {len(clientes_unicos)} (-1 se desconsiderar o 99999)") # A contagem foi feita pelo tamanho da lista de clientes únicos (que possuem Cód. de Cliente distintos)

Clientes únicos: 4326 (-1 se desconsiderar o 99999)


### 5. Vendas por bairro: quantidade de vendas por bairros/locais de entrega.

In [ ]:
df_fw = df_notas_novo

# df_por_municipio = df_fw.groupby(["Municipio"]).size() # Vendas por município
df_por_local = df_fw.groupby(["Bairro"]).size().sort_values(ascending=False)

df_por_local

# Dá pra fazer mapa de calor com isso

# CONSERTAR LOTEAMENTO BOA ESPERANCA / SANTO INÁCIO / PRIMAVERA / 

Bairro
CENTRO                      1827136
INTERIOR                     362232
APARECIDA                     77017
ITAPAGE                       67038
FATIMA                        46042
SAO FRANCISCO DE PAULA        23373
JARDIM PRIMAVERA              20782
SANTO ANTONIO                 19374
SANTO INACIO                  19034
SAO CRISTOVAO                 12698
IPIRANGA                      12069
DISTRITO INDUSTRIAL           11282
BARRIL                         8102
PANOSSO                        7496
DISTRITO DE CASTELINHO         6985
FAGUENSE                       3250
NUCLEO                         2848
LOTEAMENTO BOA ESPERANCA       1987
ITAPAJE                        1908
BOA ESPERANCA                  1881
DISTRITO DE OSVALDO CRUZ       1501
AVAMAU                         1002
LOT BOA ESPERANCA               588
LOTEAMENTO BOA ESPERNCA         391
PRIMAVERA                       150
NOVO APARECIDA                  138
BAIRRO SANTO INACIO              14
SAO PAULO            

### 6. Clientes com maiores vendas: identificação do perfil do cliente (valor de vendas, ticket médio, frequência de retorno, itens adquiridos, ...).

In [18]:
# Data mais recente no dataset para cálculo de recência
df_notas_novo['TotalItem'] = df_notas_novo['Quantidade'] * df_notas_novo['Preco']
data_analise = df_notas_novo['Data'].max()

print("--- ANÁLISE DE PERFIL: CLIENTES IDENTIFICADOS (EXCLUINDO 99999) ---")

df_clientes_identificados = df_notas_novo[df_notas_novo['Cliente'] != 99999].copy()

# 1. Selecionar os clientes com maior quantidade de vendas (Top 10)
# A 'quantidade de vendas' pode ser interpretada como o valor total vendido ou o número de compras (controles únicos).
# Vamos considerar o valor total vendido.

valor_vendas_por_cliente_identificado = df_clientes_identificados.groupby('Cliente')['TotalItem'].sum()
clientes_maior_valor_vendas = valor_vendas_por_cliente_identificado.nlargest(10)

print("\n1. Clientes com Maior Valor Total de Vendas:")
print(clientes_maior_valor_vendas)

# 2. Definir o ticket médio por cliente
# Contagem de compras únicas por cliente identificado
compras_unicas_por_cliente_identificado = df_clientes_identificados.groupby('Cliente')['Controle'].nunique()
# Ticket médio = Valor Total de Vendas / Número de Compras Únicas
ticket_medio_clientes_identificados = (valor_vendas_por_cliente_identificado / compras_unicas_por_cliente_identificado).dropna()

print("\n2. Ticket Médio por Cliente (Top 10):")
print(ticket_medio_clientes_identificados.nlargest(10))

# 3. Frequência de retorno (Número de compras únicas por cliente)
# Já calculado acima: compras_unicas_por_cliente_identificado
print("\n3. Frequência de Retorno (Número de Compras Únicas por Cliente - Top 10):")
print(compras_unicas_por_cliente_identificado.nlargest(10))

# 4. Principais itens adquiridos por esses clientes (Top 10 produtos em valor ou quantidade)
# Para os clientes identificados como um todo:

print(df_clientes_identificados)

itens_adquiridos_clientes_identificados = df_clientes_identificados.groupby('ProNom')['TotalItem'].sum().nlargest(10)

print("\n4. Principais Itens Adquiridos pelos Clientes Identificados (Top 10 por Valor Total Vendido):")
print(itens_adquiridos_clientes_identificados)

# **Bônus: Perfil RFV (Recência, Frequência, Valor) para clientes identificados**
# Recência: Dias desde a última compra

# Juntar todas as métricas em um DataFrame de perfil
df_perfil_clientes_identificados = pd.DataFrame({
    'Total_Vendas': valor_vendas_por_cliente_identificado,
    'Num_Compras_Unicas': compras_unicas_por_cliente_identificado,
    'Ticket_Medio': ticket_medio_clientes_identificados,
}).sort_values(by='Total_Vendas', ascending=False) # Ordenar pelos maiores valores de vendas

print("\n--- DataFrame de Perfil dos Clientes Identificados (Top 5) ---")
print(df_perfil_clientes_identificados.head(10))

--- ANÁLISE DE PERFIL: CLIENTES IDENTIFICADOS (EXCLUINDO 99999) ---

1. Clientes com Maior Valor Total de Vendas:
Cliente
22064    127360.6493
30907    119132.4500
5709      63955.3625
3392      59351.3998
3525      55690.3179
28466     53498.5076
14203     49402.1616
28319     43446.1331
16791     40963.4208
2755      39439.6137
Name: TotalItem, dtype: float64

2. Ticket Médio por Cliente (Top 10):
Cliente
15301    3642.600000
36557    3405.858000
20323    2568.720000
37297    2304.246667
30907    2247.782075
33979    1855.950000
31457    1751.872200
26389    1604.003400
6366     1560.317673
7957     1499.760000
dtype: float64

3. Frequência de Retorno (Número de Compras Únicas por Cliente - Top 10):
Cliente
4967     876
20355    551
33771    508
38046    503
15658    490
37797    489
36180    481
37691    455
33229    448
38468    440
Name: Controle, dtype: int64
               Data  Controle  Cliente         Bairro  ProCod  NumSeq  \
2        2024-01-02    630319    27458         CE

In [ ]:
print("\n\n--- ANÁLISE DE PERFIL: CLIENTE GENÉRICO (99999) ---")

df_cliente_99999 = df_notas_novo[df_notas_novo['Cliente'] == 99999].copy()

if not df_cliente_99999.empty:
    # 1. Valor de vendas para o cliente 99999
    valor_vendas_99999 = df_cliente_99999['TotalItem'].sum()
    print(f"\n1. Valor Total de Vendas do Cliente 99999: R$ {valor_vendas_99999:.2f}")

    # 2. Definir o ticket médio para o cliente 99999
    num_compras_unicas_99999 = df_cliente_99999['Controle'].nunique()
    ticket_medio_99999 = valor_vendas_99999 / num_compras_unicas_99999 if num_compras_unicas_99999 > 0 else 0

    print(f"\n2. Ticket Médio do Cliente 99999: R$ {ticket_medio_99999:.2f}")

    # 3. Frequência de retorno (Número de compras únicas) para o cliente 99999
    # Já calculado acima: num_compras_unicas_99999
    print(f"\n3. Frequência de Retorno (Número de Compras Únicas) do Cliente 99999: {num_compras_unicas_99999}")

    # 4. Principais itens adquiridos pelo cliente 99999 (Top 10 produtos em valor ou quantidade)
    itens_adquiridos_99999 = df_cliente_99999.groupby('ProNom')['TotalItem'].sum().nlargest(10)

    print("\n4. Principais Itens Adquiridos pelo Cliente 99999 (Top 10 por Valor Total Vendido):")
    print(itens_adquiridos_99999)

    # **Bônus: Perfil RFV para o cliente 99999**
    ultima_compra_99999 = df_cliente_99999['Data'].max()
    recencia_99999 = (data_analise - ultima_compra_99999).days

    print(f"\nRecência do Cliente 99999 (dias desde a última compra): {recencia_99999} dias")

    print("\n--- Resumo do Perfil do Cliente 99999 ---")
    print(f"Total de Vendas: R$ {valor_vendas_99999:.2f}")
    print(f"Número de Compras Únicas: {num_compras_unicas_99999}")
    print(f"Ticket Médio: R$ {ticket_medio_99999:.2f}")
    print(f"Recência (Dias): {recencia_99999}")

else:
    print("O cliente 99999 não foi encontrado no DataFrame.")

--- ANÁLISE DE PERFIL: CLIENTES IDENTIFICADOS (EXCLUINDO 99999) ---

1. Clientes com Maior Valor Total de Vendas:
Cliente
22064    127360.6493
30907    119132.4500
5709      63955.3625
3392      59351.3998
3525      55690.3179
28466     53498.5076
14203     49402.1616
28319     43446.1331
16791     40963.4208
2755      39439.6137
Name: TotalItem, dtype: float64

2. Ticket Médio por Cliente (Top 10):
Cliente
15301    3642.600000
36557    3405.858000
20323    2568.720000
37297    2304.246667
30907    2247.782075
33979    1855.950000
31457    1751.872200
26389    1604.003400
6366     1560.317673
7957     1499.760000
dtype: float64

3. Frequência de Retorno (Número de Compras Únicas por Cliente - Top 10):
Cliente
4967     876
20355    551
33771    508
38046    503
15658    490
37797    489
36180    481
37691    455
33229    448
38468    440
Name: Controle, dtype: int64

4. Principais Itens Adquiridos pelos Clientes Identificados (Top 10 por Valor Total Vendido):
ProNom
COXA SOBRECOXA FRANG

### 7. Volume de vendas: Total de faturamento de um período.